# 1. Importación de librerías y carga de datos

In [1]:
from itertools import groupby

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datos_ejemplo.csv")
df

,Región,Producto,Mes,Ventas,Unidades
0,Centro,A,Enero,305,48
1,Norte,A,Marzo,180,15
2,Centro,B,Marzo,661,8
3,Centro,B,Enero,971,14
4,Norte,A,Enero,487,23
5,Norte,A,Marzo,101,40
6,Centro,A,Febrero,489,21
7,Sur,C,Enero,665,16
8,Centro,C,Febrero,205,45
9,Centro,C,Febrero,871,18


# 2.Indexación de datos Jerárquica

In [4]:
#Crear un índice jerarquico con Región y Producto
df_jerarquico = df.set_index(["Región","Producto"]).sort_index()
print("DataFrame con índice jerárquico (Región, Producto):")
df_jerarquico

DataFrame con índice jerárquico (Región, Producto):


Mes  Ventas  Unidades
Región Producto                           
Centro A           Enero     305        48
       A         Febrero     489        21
       B           Marzo     661         8
       B           Enero     971        14
       B         Febrero     921        47
       B           Enero     501        25
       C         Febrero     205        45
       C         Febrero     871        18
       C           Enero     576        24
Norte  A           Marzo     180        15
       A           Enero     487        23
       A           Marzo     101        40
       B         Febrero     802        26
       B           Marzo     655        41
Sur    A           Marzo     369        45
       C           Enero     665        16
       C         Febrero     829        45
       C           Marzo     261        29
       C           Enero     301        15
       C           Marzo     962         1

In [5]:
#Acceder a datos específicos en el df jerarquizado
print("Acceso a datos del 'Centro':")
df_jerarquico.loc["Centro"]

Acceso a datos del 'Centro':


,Mes,Ventas,Unidades
Producto,,,
A,Enero,305,48
A,Febrero,489,21
B,Marzo,661,8
B,Enero,971,14
B,Febrero,921,47
B,Enero,501,25
C,Febrero,205,45
C,Febrero,871,18
C,Enero,576,24


In [7]:
print("Acceso a datos específicos 'Centro' y 'Producto A':")
df_jerarquico.loc[("Centro", "A")]

Acceso a datos específicos 'Centro' y 'Producto A':


Mes  Ventas  Unidades
Región Producto                           
Centro A           Enero     305        48
       A         Febrero     489        21

# 3. Agrupación de datos con groupby

In [10]:
#Agrupación básico por región
print("Ventas totales por región:")
ventas_region = df.groupby("Región")['Ventas'].sum()
ventas_region

Ventas totales por región:


Región
Centro    5500
Norte     2225
Sur       3387
Name: Ventas, dtype: int64

In [12]:
print("Estadisticas de unidades por producto:")
estadisticas = df.groupby("Producto")['Unidades'].agg(['mean', 'sum','count','min','max'])
estadisticas

Estadisticas de unidades por producto:


,mean,sum,count,min,max
Producto,,,,,
A,32.000000,192,6,15,48
B,26.833333,161,6,8,47
C,24.125000,193,8,1,45


In [16]:
print("Ventas promedio por mes:")
ventas_promedio = df.groupby("Mes")['Ventas'].mean().round(2)
ventas_promedio

Ventas promedio por mes:


Mes
Enero      543.71
Febrero    686.17
Marzo      455.57
Name: Ventas, dtype: float64

In [17]:
#Aplicar multiples funciones en varios campos a la vez a través de 'agg()'
print("Análisis completo por región:")
analisis_region = df.groupby("Región").agg({
    'Ventas' : ['mean', 'sum','count','min','max'],
    'Unidades': ['sum','max','min']
}).round(2)
analisis_region

Análisis completo por región:


Ventas                       Unidades        
          mean   sum count  min  max      sum max min
Región                                               
Centro  611.11  5500     9  205  971      250  48   8
Norte   445.00  2225     5  101  802      145  41  15
Sur     564.50  3387     6  261  962      151  45   1

# 4.Pivoteo de datos

In [21]:
# Crear una tabla pivoteada (dinámica): Regiones vs Productos (suma de ventas)
print("Tabla pivote - Ventas por Región y Producto")
pivot_ventas = df.groupby(['Región','Producto'])['Ventas'].sum().unstack(fill_value=0)
pivot_ventas

Tabla pivote - Ventas por Región y Producto


Producto,A,B,C
Región,,,
Centro,794,3054,1652
Norte,768,1457,0
Sur,369,0,3018


In [22]:
print("Tabla pivote - Unidades por Región y Mes")
pivot_unidades = df.groupby(["Mes","Región"])["Unidades"].sum().unstack(fill_value=0)
pivot_unidades

Tabla pivote - Unidades por Región y Mes


Región,Centro,Norte,Sur
Mes,,,
Enero,111,23,31
Febrero,131,26,45
Marzo,8,96,75


# 5. Despivoteo de datos

In [26]:
#Despivotear tabla 'pivot_ventas'
print("Despivoteo de la tabla de ventas")
despivot_ventas = pivot_ventas.reset_index().melt(
    id_vars = ["Región"],
    var_name = "Producto",
    value_name = "Ventas_total"
)
despivot_ventas[despivot_ventas['Ventas_total']>0]

Despivoteo de la tabla de ventas


,Región,Producto,Ventas_total
0,Centro,A,794
1,Norte,A,768
2,Sur,A,369
3,Centro,B,3054
4,Norte,B,1457
6,Centro,C,1652
8,Sur,C,3018
